In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [12]:
nltk.download('wordnet')
nltk.download('omw-1.4')
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


51 documents
10 classes ['Location', 'courses', 'fest', 'goodbye', 'greeting', 'information', 'options', 'placement', 'seats', 'thanks']
81 unique lemmatized words ["'s", ',', '.', 'about', 'address', 'and', 'anyone', 'are', 'available', 'awesome', 'b', 'be', 'by', 'bye', 'can', 'chatting', 'college', 'collge', 'come', 'company', 'computer', 'contact', 'could', 'course', 'day', 'do', 'engineering', 'event', 'fest', 'for', 'function', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'information', 'intake', 'is', 'know', 'later', 'list', 'location', 'mba', 'me', 'next', 'nice', 'of', 'offer', 'offered', 'parampara', 'place', 'placement', 'provide', 'record', 'science', 'seat', 'see', 'support', 'tech', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'want', 'what', 'which', 'you']


In [13]:
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

In [14]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-14-8451cd400d46>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [15]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [16]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
11/11 [==============================] - 2s 8ms/step - loss: 2.4081 - accuracy: 0.1176
Epoch 2/200
11/11 [==============================] - 0s 4ms/step - loss: 2.2939 - accuracy: 0.0588
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 2.2574 - accuracy: 0.0980
Epoch 4/200
11/11 [==============================] - 0s 5ms/step - loss: 2.2160 - accuracy: 0.1961
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 2.1656 - accuracy: 0.1765
Epoch 6/200
11/11 [==============================] - 0s 3ms/step - loss: 2.1205 - accuracy: 0.2745
Epoch 7/200
11/11 [==============================] - 0s 4ms/step - loss: 1.9887 - accuracy: 0.3333
Epoch 8/200
11/11 [==============================] - 0s 5ms/step - loss: 2.0124 - accuracy: 0.3137
Epoch 9/200
11/11 [==============================] - 0s 7ms/step - loss: 1.7826 - accuracy: 0.4510
Epoch 10/200
11/11 [==============================] - 0s 4ms/step - loss: 1.7464 - accuracy: 0.4902
Epoch 11/